In [1]:
import numpy as np
import pandas as pd
from fairness import *
from sklearn.metrics import roc_curve, auc
import pickle

with open('sens_attr_dict_test.pkl', 'rb') as file:
    sens_attr_dict_test = pickle.load(file)


LATEX = False


task_names= {'Walmart-Amazon': r'\walamz',
'Amazon-Google': r'\amzgog',
'DBLP-GoogleScholar': r'\DBLPGogS',
'DBLP-ACM': r'\DBLPACM',
'iTunes-Amazon': r'\itunamz',
'Fodors-Zagat': r'\FodorZag',
'Beer': r'\Beer'}




model_names = {'DITTO':r'\ditto',
'HierGAT':r'\hiergat',
'deepmatcher':r'\dmatcher',
'HierMatcher':r'\hmatcher',
'EMTransformer':r'\EMT',}

path_base = os.getcwd()

tasks = ['Fodors-Zagat', 'DBLP-GoogleScholar', 'iTunes-Amazon', 'Walmart-Amazon', 'Amazon-Google','Beer','DBLP-ACM']
models = ['deepmatcher', 'DITTO', 'EMTransformer', 'HierGAT','HierMatcher']


    
for model in models:
    if LATEX: 
        print(r'\multirow{7}{*}{\rotatebox[origin=c]{90}{ \small ' + model_names[model] + r'~}}')

    for task in tasks:
        # if task+'_'+model in skip_combinations: continue


        df_test = pd.read_csv(path_base +'/DATA_VLDB/'+ task  + '/test.csv')
        scores_test = pd.read_csv(path_base+'/VLDB_RES/'+task+'_'+model+'/score_test.csv')
        sens_test = sens_attr_dict_test[task]

        for x in list(scores_test.columns):
            if len(np.unique(scores_test[x])) >2: 
                prob_test = scores_test[x]
            else:
                y_test = scores_test[x]
            

        sens_array = sens_test
        prob_array = prob_test
        y_array = y_test


        prob_minor = prob_array[sens_array ==1]
        prob_major = prob_array[sens_array ==0]


        y_minor = y_array[sens_array ==1]
        y_major = y_array[sens_array ==0]


        DP, EO, EOD =[], [], []
        PR_minor_all =[]
        TPR_minor_all =[]

        PR_major_all =[]
        TPR_major_all =[]



        fpr, tpr, _ = roc_curve(y_array, prob_array)
        auc_all = 100*auc(fpr, tpr)


        fpr, tpr, _ = roc_curve(y_major, prob_major)
        auc_major = 100*auc(fpr, tpr) 


        N = 500
        for theta in np.linspace(0, 1, N):

            y_pred = np.array([1 if score > theta else 0 for score in prob_minor])
            tn, fp, fn, tp = confusion_matrix(y_minor, y_pred).ravel()
            tpr = tp / (tp + fn)
            pr = (tp + fp) / len(y_minor)
            fpr = fp / (fp + tn)
            PR_minor, FPR_minor, TPR_minor = pr, fpr, tpr
            PR_minor_all.append(PR_minor)
            TPR_minor_all.append(TPR_minor)

            y_pred = np.array([1 if score > theta else 0 for score in prob_major])
            tn, fp, fn, tp = confusion_matrix(y_major, y_pred).ravel()
            tpr = tp / (tp + fn)
            pr = (tp + fp) /len(y_major)
            fpr = fp / (fp + tn)
            PR_major, FPR_major, TPR_major = pr, fpr, tpr
            PR_major_all.append(PR_major)
            TPR_major_all.append(TPR_major)

            EOD.append((np.abs(TPR_major - TPR_minor) + np.abs(FPR_major - FPR_minor)))
            EO.append(np.abs(TPR_major - TPR_minor))
            DP.append(np.abs(PR_major-PR_minor))

        DSP_EOD = 100*np.average(EOD) 
        DSP_EO = 100*np.average(EO)
        DSP_DP = 100*np.average(DP)
        Frac = np.linspace(0, 1, N)
        if not LATEX: 
        
            print(task_names[task], model, round(DSP_DP,2), round(DSP_EO,2), round(DSP_EOD,2), round(auc_all,2))



            for i in [50,250,500*8//10,500*95//100 ]:
                print('\t',round(Frac[i],2),round(100*(DP[i]),2),round(100*EO[i],2), round(100*EOD[i],2))
            print()

        
        if LATEX: 
            print('& '+task_names[task], end =' & ')
            print(round(auc_all,2),r' & \multicolumn{1}{!{\vrule width 1pt}c}{',round(DSP_DP,2), end= r'} & ')#,round(DSP_EO,2),' & ', round(DSP_EOD,2), end =' & ')
            for i in [50,250,500*8//10,500*95//100 ]:
                if i == 500*95//100:
                    print(round(100*(DP[i]),2),end =r' &')
                else:
                    print(round(100*(DP[i]),2),end =' & ')
            
            print(r'  \multicolumn{1}{!{\vrule width 1pt}c}{',round(DSP_EO,2), end= r'} & ')#,round(DSP_EO,2),' & ', round(DSP_EOD,2), end =' & ')
            for i in [50,250,500*8//10,500*95//100 ]:
                print(round(100*(EO[i]),2),end =' & ')


            print(r'  \multicolumn{1}{!{\vrule width 1pt}c}{',round(DSP_EOD,2), end= r'} & ')#,round(DSP_EO,2),' & ', round(DSP_EOD,2), end =' & ')
            for i in [50,250,500*8//10,500*95//100 ]:
                if i == 500*95//100:
                    print(round(100*(EOD[i]),2),end =r' \\')
                else:
                    print(round(100*(EOD[i]),2),end =' & ')
            
            print()
            print(r'\midrule')

            print('\n\n')

\FodorZag deepmatcher 2.86 5.51 5.77 100.0
	 0.1 5.01 0.0 2.19
	 0.5 2.45 5.26 5.26
	 0.8 2.45 5.26 5.26
	 0.95 1.92 15.79 15.79

\DBLPGogS deepmatcher 6.15 11.53 11.76 98.41
	 0.1 6.58 7.58 7.79
	 0.5 6.61 10.76 10.96
	 0.8 7.38 15.13 15.31
	 0.95 1.8 7.87 7.96

\itunamz deepmatcher 13.84 17.98 23.88 96.97
	 0.1 21.12 9.09 26.16
	 0.5 16.19 21.02 28.34
	 0.8 9.18 14.77 14.77
	 0.95 7.02 25.0 25.0

\walamz deepmatcher 3.74 13.8 14.79 82.4
	 0.1 3.19 11.39 11.48
	 0.5 4.38 16.08 17.19
	 0.8 3.66 11.69 12.45
	 0.95 0.83 4.1 4.16

\amzgog deepmatcher 8.94 4.36 8.19 89.02
	 0.1 13.52 13.96 20.66
	 0.5 9.61 1.35 5.31
	 0.8 8.04 2.93 5.73
	 0.95 2.3 3.38 3.86

\Beer deepmatcher 25.77 23.0 37.22 95.73
	 0.1 40.27 0.0 29.17
	 0.5 26.72 26.67 40.95
	 0.8 20.05 4.44 18.73
	 0.95 6.67 11.11 15.87

\DBLPACM deepmatcher 3.63 1.55 1.77 99.57
	 0.1 4.5 1.05 1.51
	 0.5 3.91 1.57 1.72
	 0.8 3.95 0.77 0.98
	 0.95 0.82 7.76 7.76

\FodorZag DITTO 3.09 7.99 8.56 99.97
	 0.1 5.01 0.0 2.19
	 0.5 2.45 5.26 5.